In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.preprocessing import image

PATHS


In [ ]:
dataset_path = '/content/drive/MyDrive/skin disease dataset'
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')

PARAMETERS

In [ ]:
img_height, img_width = 128, 128
batch_size = 256
epochs = 30

IMAGE DATA GENERATORS

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 15557 images belonging to 23 classes.
Found 4012 images belonging to 23 classes.


NUMBER OF CLASSES

In [ ]:
num_classes = len(train_generator.class_indices)
class_names = list(train_generator.class_indices.keys())

MODEL ARCHITECTURE

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


COMPILE THE MODEL

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

TRAIN THE MODEL

In [ ]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
 1/61 ━━━━━━━━━━━━━━━━━━━━ 1:26:19 86s/step - accuracy: 0.0273 - loss: 3.1634

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"\nTest Accuracy: {test_accuracy:.2f}")

126/126 ━━━━━━━━━━━━━━━━━━━━ 54s 427ms/step - accuracy: 0.4164 - loss: 4.6631

Test Accuracy: 0.29


In [ ]:
# Predict
y_pred = model.predict(test_generator)
y_true = test_generator.classes
y_pred_classes = np.argmax(y_pred, axis=1)

126/126 ━━━━━━━━━━━━━━━━━━━━ 54s 425ms/step


In [ ]:
# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

In [ ]:
# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[280   3   2   1   1   1   0   0   0   6   0   0   0   0   4   0   4   1
    3   1   0   1   4]
 [ 13  77   5   2   3  13   1   0   6  10   0   6   5   2  30   1  48  12
   20   1  10   1  22]
 [  6   2  10   2   1  19   1   0   1   8   3   2  13   0  12   1   9   7
   11   1   3   4   7]
 [  2   7   0   9   1   6   1   1   3   7   1   2   6   1  13   0  11   9
   13   1   3   3  13]
 [  6   4   2   2   4   8   0   1   2   2   0   3   4   1  17   2   3   2
   12   0   2   0   6]
 [ 13   8   5   0   1 120   3   0   2   3   3   2  24   1  41   2  16   9
   38   1   2   2  13]
 [  3   5   2   0   1   9  19   0   0   7   1   2   3   0  17   2   2   3
   18   1   0   0   6]
 [  1   1   1   0   2   2   0  15   1   2   0   0  14   0   5   0   8   0
    6   0   1   0   1]
 [  1  11   1   1   0   2   0   1   6   2   1   1   5   1  12   0  15   3
   10   0   3   1  25]
 [ 14   4   2   0   0  16   3   0   8  19   1   4   2   1  27   4   9   4
   13   0   0   4   8]
 [  7   4  

In [ ]:
y_true_bin = label_binarize(y_true, classes=list(range(num_classes)))

fpr = {}
tpr = {}
roc_auc = {}

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8, 6))
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Multi-class ROC Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()


In [ ]:
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    print("Predicted Disease:", class_names[predicted_class])


predict_image("/content/drive/MyDrive/skin disease dataset/test/Light Diseases and Disorders of Pigmentation/hydroa-aestivale-2.jpg")
predict_image("/content/drive/MyDrive/skin disease dataset/test/Atopic Dermatitis Photos/05Atopic12060415.jpg")
predict_image("/content/drive/MyDrive/skin disease dataset/test/Nail Fungus and other Nail Disease/alopecia-areata-39.jpg")
predict_image("/content/drive/MyDrive/skin disease dataset/test/Poison Ivy Photos and other Contact Dermatitis/allergic-contact-dermatitis-17.jpg")
predict_image("/content/drive/MyDrive/skin disease dataset/test/Tinea Ringworm Candidiasis and other Fungal Infections/candida-groin-2.jpg")